In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/My\ Drive/TryTextAttackAgain

/content/drive/My Drive/TryTextAttackAgain


In [3]:
!ls 

BaseLoading.ipynb		      InferSent_snli_pretrained_GLOVE
corpus.py			      multi_nli_train_word_freq.csv
DefendBertGuide.ipynb		      preprocess_datasets
detect_adversarial.py		      prioritized.py
EvalBert.ipynb			      __pycache__
freq_attacks_mnli.py		      RunFull
freq_attacks_snli.py		      snli_train_word_freq.csv
freq_stats.py			      TextAttack
InferSent.ipynb			      TrainedBertModels
inferSentMultiModel.py		      TrainNotebooks
InferSent_multi_nli_pretrained_GLOVE  wandb
inferSentSNLIModel.py


In [4]:
#!git clone https://github.com/QData/TextAttack.git

In [5]:

%cd TextAttack/

/content/drive/My Drive/TryTextAttackAgain/TextAttack


In [6]:
!pip install pyarrow>=0.16.0 --force-reinstall
!python3 setup.py install

ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: textattack 0.2.4 has requirement numpy<1.19.0, but you'll have numpy 1.19.1 which is incompatible.
ERROR: flair 0.6 has requirement pytest>=5.3.2, but you'll have pytest 3.6.4 which is incompatible.
running install
running bdist_egg
running egg_info
writing textattack.egg-info/PKG-INFO
writing dependency_links to textattack.egg-info/dependency_links.txt
writing entry points to textattack.egg-info/entry_points.txt
writing requirements to textattack.egg-info/requires.txt
writing top-level names to textattack.egg-info/top_level.txt
writing manifest file 'textattack.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
run

In [7]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.8.1          
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.2.10         
astor                         0.8.1          
astropy                       4.0.1.post1    
astunparse                    1.6.3          
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.1.0         
audioread                     2.1.8          
autograd                      1.3            
Babel                         2.8.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bert-score                    0.3.5          
bleach                        3.1.5          
blis                          0.4.

In [8]:
!ls ../

BaseLoading.ipynb		      InferSent_snli_pretrained_GLOVE
corpus.py			      multi_nli_train_word_freq.csv
DefendBertGuide.ipynb		      preprocess_datasets
detect_adversarial.py		      prioritized.py
EvalBert.ipynb			      __pycache__
freq_attacks_mnli.py		      RunFull
freq_attacks_snli.py		      snli_train_word_freq.csv
freq_stats.py			      TextAttack
InferSent.ipynb			      TrainedBertModels
inferSentMultiModel.py		      TrainNotebooks
InferSent_multi_nli_pretrained_GLOVE  wandb
inferSentSNLIModel.py


In [9]:
import textattack

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [10]:
!ls ../TrainedBertModels/bert-base-uncased-snli

1597448743.868051				   README.md
attack						   special_tokens_map.json
config.json					   tokenizer_config.json
events.out.tfevents.1597448743.a5d2f71e29f2.534.0  train_args.json
log.txt						   vocab.txt
pytorch_model.bin


# Evaluating model

In [11]:
!textattack eval --model-from-huggingface  ../TrainedBertModels/bert-base-uncased-snli  --dataset-from-nlp snli --num-examples 10000 --dataset_split test --shuffle False

2020-08-26 11:09:34.320913: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/bert-base-uncased-snli
textattack: Loading nlp dataset snli, split test.
textattack: Got 10000 predictions.
textattack: Successes 8916/10000 (89.16%)


# Create frequency dictionary

In [12]:
import string
import nltk
import nlp
import csv
import os
import math 
from textattack.shared.utils import words_from_text
import numpy as np

def get_tokenize_sentences(premises,hypothesis):
  all_sentences = []

  for i in zip(premises,hypothesis):
    st = i[0] + " " + i[1]
    tokens = words_from_text(st)
    all_sentences.append(tokens)
  
  return all_sentences

def return_multi_nli_train_dict(dataset):

  train_data = nlp.load_dataset(dataset)['train']

  train_dict = {'premise':[],'hypothesis':[]}

  perc_ = math.ceil(len(train_data)*0.15)
  print("----EVAL DATASET USING : " + str(perc_) + "\n")

  eval_indices = np.sort(np.random.RandomState(21).choice(np.arange(len(train_data)),size=perc_,))
  train_indices = np.delete(np.arange(len(train_data)),eval_indices)

  train_dict['premise'] = np.array(train_data['premise'])[train_indices] 
  train_dict['hypothesis'] = np.array(train_data['hypothesis'])[train_indices]

  return train_dict

def create_freq_corpus(dataset,csvname):


  if dataset is 'multi_nli':
   print('MULTI') 
   train_data = return_multi_nli_train_dict(dataset)

  else:

    train_data = nlp.load_dataset(dataset)['train']

  
  all_sentences = get_tokenize_sentences(train_data['premise'],train_data['hypothesis'])

  freq_dict = {}

  for sentence in all_sentences:
    for token in sentence:
      if token  in freq_dict:
        freq_dict[token] += 1
      else:
        freq_dict[token] = 1 

  print("WRITE")
  with open(csvname, 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in freq_dict.items():
      writer.writerow([key, value])
  

  return freq_dict

def load_csv(filename):

  with open(filename) as csv_file:
    reader = csv.reader(csv_file)
    freq_dict = dict(reader)
  
  return freq_dict


# SNLI

In [13]:
snli_file_name = '../snli_train_word_freq.csv'
snli_dataset_name = 'snli'
#snli_freq_dict = create_freq_corpus(snli_dataset_name,snli_file_name)
snli_freq_dict = load_csv(snli_file_name)

In [14]:
snli_freq_dict['the']

'370390'

In [15]:
snli_freq_dict['The']

'164519'

# To get frequency statistics

In [16]:
import pandas as pd
from ast import literal_eval
import numpy as np
import math 

def phi_value(freq):
  return math.log(1+freq)

def get_phi_values(words,freq_dict,in_dict=False):

  if in_dict:
    return [phi_value(int(freq_dict[word])) for word in words if word in freq_dict]

  return [phi_value(int(freq_dict[word])) if word in freq_dict else 0 for word in words]


def get_attack_freq_stats(filename,freq_dict):
  
  attack_data = pd.read_csv(filename)

  premises = [i['premise'] for _, i in attack_data.iterrows()] 
  attacked_hypo = [i['attacked_hypothesis'] for _, i in attack_data.iterrows()] 
  original_hypo = [i['original_hypothesis'] for _, i in attack_data.iterrows()] 
  modifiable_indices = [literal_eval(i['modifiable_indices']) for _, i in attack_data.iterrows()] 
  modified_indices = [literal_eval(i['modified_indices']) for _, i in attack_data.iterrows()] 

  attacked_sentences = get_tokenize_sentences(premises,attacked_hypo)
  original_sentences = get_tokenize_sentences(premises,original_hypo)
  
  replaceable_words =[]
  replaced_words = []
  sub_words = []

  for i in range(len(original_sentences)):
    replaceable_words.append(np.array(original_sentences[i])[modifiable_indices[i]])
    replaced_words.append(np.array(original_sentences[i])[modified_indices[i]])
    sub_words.append(np.array(attacked_sentences[i])[modified_indices[i]])

  replaceable_words_freqs = []
  replaced_words_freqs = []
  sub_words_freqs = []
  sub_ovv_freqs = []

  for i in range(len(replaceable_words)):
    replaceable_words_freqs = np.concatenate((replaceable_words_freqs,get_phi_values(replaceable_words[i],freq_dict)))
    replaced_words_freqs = np.concatenate((replaced_words_freqs,get_phi_values(replaced_words[i],freq_dict)))
    sub_words_freqs = np.concatenate((sub_words_freqs,get_phi_values(sub_words[i],freq_dict)))
    sub_ovv_freqs = np.concatenate((sub_ovv_freqs,get_phi_values(sub_words[i],freq_dict, True)))

  print("MEAN:\t" + str(np.mean(replaceable_words_freqs)) + "\t STD:\t" + str(np.std(replaceable_words_freqs)))
  print("MEAN:\t" + str(np.mean(replaced_words_freqs)) + "\t STD:\t" + str(np.std(replaced_words_freqs)))
  print("MEAN:\t" + str(np.mean(sub_words_freqs)) + "\t STD:\t" + str(np.std(sub_words_freqs)))
  print("MEAN:\t" + str(np.mean(sub_ovv_freqs)) + "\t STD:\t" + str(np.std(sub_ovv_freqs)))

In [17]:
!ls ../TrainedBertModels/bert-base-uncased-snli

1597448743.868051				   README.md
attack						   special_tokens_map.json
config.json					   tokenizer_config.json
events.out.tfevents.1597448743.a5d2f71e29f2.534.0  train_args.json
log.txt						   vocab.txt
pytorch_model.bin


# Attacks

In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/bert-base-uncased-snli --recipe random_nli  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/bert-base-uncased-snli

In [ ]:
get_attack_freq_stats('../TrainedBertModels/bert-base-uncased-snli/attack/random_nli_2000_attacked.csv',snli_freq_dict)

In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/bert-base-uncased-snli/ --attack-from-class ../prioritized Priotirized  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/bert-base-uncased-snli/

In [ ]:
get_attack_freq_stats('../TrainedBertModels/bert-base-uncased-snli/attack/prioritized_Priotirized_2000_attacked.csv',snli_freq_dict)

In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/bert-base-uncased-snli --recipe textfooler  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/bert-base-uncased-snli

In [ ]:
get_attack_freq_stats('../TrainedBertModels/bert-base-uncased-snli/attack/textfooler_2000_attacked.csv',snli_freq_dict)

In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/bert-base-uncased-snli --recipe pwws_snli  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/bert-base-uncased-snli

In [ ]:
get_attack_freq_stats('../TrainedBertModels/bert-base-uncased-snli/attack/pwws_snli_2000_attacked.csv',snli_freq_dict)

In [26]:
!textattack attack --attack_output_snli ../TrainedBertModels/bert-base-uncased-snli --recipe alzantot_gpt2  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/bert-base-uncased-snli

Streaming output truncated to the last 5000 lines.
[Succeeded / Failed / Total] 804 / 454 / 1426:  71% 1426/2000 [1:48:30<43:40,  4.57s/it]--------------------------------------------- Result 1427 ---------------------------------------------
Contradiction (100%) --> Entailment (93%)

Premise: A police officer standing next his police motorbike holding a yellow safety jacket.
Hypothesis: A man driving a car.

Premise: A police officer standing next his police motorbike holding a yellow safety jacket.
Hypothesis: A man motorists a automobiles.


[Succeeded / Failed / Total] 805 / 454 / 1427:  71% 1427/2000 [1:48:33<43:35,  4.56s/it]--------------------------------------------- Result 1428 ---------------------------------------------
Entailment (98%) --> Contradiction (92%)

Premise: Little boy peering into a glass display case.
Hypothesis: a boy looks in a case

Premise: Little boy peering into a glass display case.
Hypothesis: a bloke looks in a case


[Succeeded / Failed / Total] 806

In [27]:
get_attack_freq_stats('../TrainedBertModels/bert-base-uncased-snli/attack/alzantot_gpt2_2000_attacked.csv',snli_freq_dict)

MEAN:	8.691000118365409	 STD:	2.5487802646511137
MEAN:	8.618114721286034	 STD:	2.3445300741645423
MEAN:	3.4944132431237223	 STD:	3.3832465296553433
MEAN:	4.831175500834808	 STD:	3.060546403871835


# Frequency Attacks

In [28]:
!textattack attack --attack_output_snli ../TrainedBertModels/bert-base-uncased-snli/ --attack-from-class ../freq_attacks_snli FrequencyR  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/bert-base-uncased-snli/

2020-08-26 14:31:54.786987: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 665, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/__init__.py", line 27, in <module>
    from tensorflow.python.keras import models
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/models.py", line 26, in <module>
    from tensorflow.python.keras.engine import functional
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py", line 38, in <module>
    from tensorflow.python.keras.engine import training as training_lib
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line

In [ ]:
get_attack_freq_stats('../TrainedBertModels/bert-base-uncased-snli/attack/freq_attacks_snli_FrequencyR_2000_attacked.csv',snli_freq_dict)

MEAN:	10.164118101802039	 STD:	2.7852977263496927
MEAN:	10.162575874147354	 STD:	2.7446926793316426
MEAN:	0.11099752084406923	 STD:	0.620665107757422
MEAN:	2.296046655398195	 STD:	1.7180298119283506


In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/bert-base-uncased-snli --attack-from-class ../freq_attacks_snli FrequencyP  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/bert-base-uncased-snli

Streaming output truncated to the last 5000 lines.


[Succeeded / Failed / Total] 946 / 368 / 1489:  74% 1489/2000 [02:38<00:54,  9.38it/s]--------------------------------------------- Result 1490 ---------------------------------------------
Contradiction (99%) --> [SKIPPED]

Premise: Many women trying out and looking at laptops in a Walmart.
Hypothesis: Women are shopping for Dell Computers.


[Succeeded / Failed / Total] 946 / 368 / 1490:  74% 1490/2000 [02:38<00:54,  9.39it/s]Try loading SNLI training Corpus
--------------------------------------------- Result 1491 ---------------------------------------------
Contradiction (98%) --> [FAILED]

Premise: A man with a plaid shirt stands with his hand on hip at an opening of a large tree.
Hypothesis: a man at the opening of a small bush


[Succeeded / Failed / Total] 946 / 369 / 1491:  75% 1491/2000 [02:38<00:54,  9.38it/s]--------------------------------------------- Result 1492 ---------------------------------------------
Entailment

In [ ]:
get_attack_freq_stats('../TrainedBertModels/bert-base-uncased-snli/attack/freq_attacks_snli_FrequencyP_2000_attacked.csv',snli_freq_dict)

MEAN:	10.164118101802039	 STD:	2.7852977263496927
MEAN:	10.180038561255428	 STD:	2.7334022327342393
MEAN:	0.10909409886105365	 STD:	0.6076223285866549
MEAN:	2.3087668060502367	 STD:	1.6537775683978773


In [ ]:
!ls ../TrainedBertModels/bert-base-uncased-snli/attack

alzantot_gpt2_2000_attacked.csv
freq_attacks_snli_FrequencyP_2000_attacked.csv
freq_attacks_snli_FrequencyR_2000_attacked.csv
prioritized_Priotirized_2000_attacked.csv
pwws_snli_2000_attacked.csv
random_nli_2000_attacked.csv
random_nlieval_whole_attacked.csv
textfooler_2000_attacked.csv


# Defense Models

In [ ]:
import argparse
import copy
import importlib
import json
import os
import time
import math 

import textattack

from textattack.commands.attack.attack_args import (
    ATTACK_RECIPE_NAMES,
    BLACK_BOX_TRANSFORMATION_CLASS_NAMES,
    CONSTRAINT_CLASS_NAMES,
    GOAL_FUNCTION_CLASS_NAMES,
    HUGGINGFACE_DATASET_BY_MODEL,
    SEARCH_METHOD_CLASS_NAMES,
    TEXTATTACK_DATASET_BY_MODEL,
    WHITE_BOX_TRANSFORMATION_CLASS_NAMES,
)

# The split token allows users to optionally pass multiple arguments in a single
# parameter by separating them with the split token.
ARGS_SPLIT_TOKEN = "^"

def _cb(s):
  return textattack.shared.utils.color_text(str(s), color="blue", method="ansi")

def phi_value(freq):
    return math.log(1+freq)

def load_module_from_file(file_path):
    """Uses ``importlib`` to dynamically open a file and load an object from
    it."""
    temp_module_name = f"temp_{time.time()}"
    colored_file_path = textattack.shared.utils.color_text(
        file_path, color="blue", method="ansi"
    )
    textattack.shared.logger.info(f"Loading module from `{colored_file_path}`.")
    spec = importlib.util.spec_from_file_location(temp_module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

def parse_model(model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

    if model_from_file:
        # Support loading the model from a .py file where a model wrapper
        # is instantiated.
        colored_model_name = textattack.shared.utils.color_text(
            model_from_file, color="blue", method="ansi"
        )
        textattack.shared.logger.info(
            f"Loading model and tokenizer from file: {colored_model_name}"
        )
        if ARGS_SPLIT_TOKEN in model_from_file:
            model_file, model_name = model_from_file.split(ARGS_SPLIT_TOKEN)
        else:
            _, model_name = model_from_file, "model"
        try:
            model_module = load_module_from_file(model_from_file)
            print("))))HERE(((())))")
            print(model_module)
        except Exception:
            raise ValueError(f"Failed to import file {model_from_file}")
        try:
            model = getattr(model_module, model_name)
        except AttributeError:
            raise AttributeError(
                f"``{model_name}`` not found in module {model_from_file}"
            )

        if not isinstance(model, textattack.models.wrappers.ModelWrapper):
            
            if isinstance(model, textattack.models.NLINet):
                
                model = textattack.models.wrappers.PyTorchModelWrapper(
                    model, model.encoder, batch_size=model_batch_size
                )
                
            else:
                raise TypeError(
                    "Model must be of type "
                    f"``textattack.models.ModelWrapper``, got type {type(model)}"
                )

          
    elif (model in HUGGINGFACE_DATASET_BY_MODEL) or model_from_huggingface:
        # Support loading models automatically from the HuggingFace model hub.
        import transformers

        model_name = (
            HUGGINGFACE_DATASET_BY_MODEL[args.model][0]
            if (model in HUGGINGFACE_DATASET_BY_MODEL)
            else model_from_huggingface
        )

        if ARGS_SPLIT_TOKEN in model_name:
            model_class, model_name = model_name
            model_class = eval(f"transformers.{model_class}")
        else:
            model_class, model_name = (
                transformers.AutoModelForSequenceClassification,
                model_name,
            )
        colored_model_name = textattack.shared.utils.color_text(
            model_name, color="blue", method="ansi"
        )
        textattack.shared.logger.info(
            f"Loading pre-trained model from HuggingFace model repository: {colored_model_name}"
        )
        model = model_class.from_pretrained(model_name)
        tokenizer = textattack.models.tokenizers.AutoTokenizer(model_name)
        model = textattack.models.wrappers.HuggingFaceModelWrapper(
            model, tokenizer, batch_size=model_batch_size
        )
    elif model in TEXTATTACK_DATASET_BY_MODEL:
        # Support loading TextAttack pre-trained models via just a keyword.
        model_path, _ = TEXTATTACK_DATASET_BY_MODEL[model]
        model = textattack.shared.utils.load_textattack_model_from_path(
            model, model_path
        )
        # Choose the approprate model wrapper (based on whether or not this is
        # a HuggingFace model).
        if isinstance(
            model, textattack.models.helpers.BERTForClassification
        ) or isinstance(model, textattack.models.helpers.T5ForTextToText):
            model = textattack.models.wrappers.HuggingFaceModelWrapper(
                model, model.tokenizer, batch_size=model_batch_size
            )
        else:
            model = textattack.models.wrappers.PyTorchModelWrapper(
                model, model.tokenizer, batch_size=model_batch_size
            )
    else:
      raise ValueError("Nothing else supported")
    

    return model

def parse_defense_from_class(defense_from_class=None,freq_dict=None,delta=10,type='discrete',gamma='0.1',candidates=8,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

    model = parse_model(model_from_file, model,model_from_huggingface,model_batch_size)
    
    if defense_from_class:
      import sys
      path = defense_from_class[0]
      path_class = path.rsplit('/')
      folder_path =  path_class[0] + '/'
      file = path_class[1]

      sys.path.insert(0,folder_path)
      module = importlib.import_module(file)
      my_class = getattr(module, defense_from_class[1])

      return my_class.build(model,freq_dict,delta=delta,type=type,gamma=gamma,candidates=candidates)
    else:
      raise ValueError("currently only attack from class is supported")

In [ ]:
import sys
sys.path.insert(0,"../")

In [ ]:
import scipy
import torch
import pandas as pd
from collections import deque
import corpus
import tqdm
import time
import textattack

logger = textattack.shared.logger

column_encoder = {"Entailment":0, "Neutral":1, "Contradiction":2}
reversed_encoder = {0:"Entailment", 1:"Neutral", 2:"Contradiction"}

def parse_log_manager():
    # Create logger
    attack_log_manager = textattack.loggers.AttackLogManager()

    '''
    # Get current time for file naming
    timestamp = time.strftime("%Y-%m-%d-%H-%M")

    # Get default directory to save results
    current_dir = os.path.dirname(os.path.realpath(__file__))
    outputs_dir = os.path.join(
        current_dir, os.pardir, os.pardir, os.pardir, "outputs", "attacks"
    )
    out_dir_txt = out_dir_csv = os.path.normpath(outputs_dir)

    # Get default txt and csv file names
    filename_txt = f"_{timestamp}.txt"
    filename_csv = f"_{timestamp}.csv"
    
    

    # in case directory doesn't exist
    if not os.path.exists(out_dir_txt):
        os.makedirs(out_dir_txt)
    if not os.path.exists(out_dir_csv):
        os.makedirs(out_dir_csv)

    attack_log_manager.add_output_file(os.path.join(out_dir_txt, filename_txt))
    '''
    #attack_log_manager.enable_visdom()
    #attack_log_manager.enable_wandb()
    attack_log_manager.enable_stdout()
    return attack_log_manager

def evaluate_set(attacked_dataset,attack,logger=True):

  num_remaining_attacks = len(attacked_dataset)
  worklist = deque(range(0, len(attacked_dataset)))
  start_time = time.time()
  worklist_tail = worklist[-1]

  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)
  #pbar = tqdm.autonotebook.tqdm(total=num_remaining_attacks)
  attack_log_manager = parse_log_manager()


  num_results = 0
  num_failures = 0
  num_successes = 0

  load_time = time.time()

  probabilities = []
  for result in attack.attack_dataset(attacked_dataset, indices=worklist):

    #or_label = result.original_result.output
    #initial_score = result.original_result.raw_output[or_label]
    #pertubated_score = result.perturbed_result.raw_output[or_label]
    initial_score = 1-result.original_result.score
    pertubated_score = 1-result.perturbed_result.score
    probabilities.append(initial_score-pertubated_score)

    if logger:
      attack_log_manager.log_result(result)

      print("\n")
    pbar.update(1)
    
    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )
    
  pbar.close()

  # Enable summary stdout
  
  if logger:
    attack_log_manager.log_summary()
    attack_log_manager.flush()
  print()
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  return num_successes, num_failures, probabilities



class EvalDefenseModel():

  def __init__(
      self,
      defense_from_class,
      model_from_file=None, 
      model=None,
      model_from_huggingface=None,
      model_batch_size=32,
      delta = 10,
      gamma = 1,
      type = 'discrete',
      freq_dict = None,
      data_file = None,
      data_name = "snli",
      candidates = 8,
      logger = True,
      only_attack = False,
      only_original = False,
      #indices_to_modify=None,
      #shifted_idxs=True,
    ):

    # Get attack
    attack = parse_defense_from_class(defense_from_class,freq_dict,delta,type,gamma,candidates,model_from_file, model,model_from_huggingface,model_batch_size)
    
    #Get datasets
    attacked_data = pd.read_csv(data_file,usecols=["premise",'attacked_hypothesis', 'modified_indices' , "attacked_label"])
    original_data = pd.read_csv(data_file,usecols=["premise",'original_hypothesis', "original_label"])
    or_labels = original_data["original_label"]

    to_remove = []
    for index,x in attacked_data.iterrows():
      if x['modified_indices'] == '[]' or or_labels[index]==x['attacked_label']:
        to_remove.append(index)
    attacked_data = attacked_data.drop(to_remove)
    attacked_data = attacked_data.drop(['modified_indices'],axis=1)

    attacked_data["attacked_label"] = [column_encoder[i] for i in attacked_data["attacked_label"] ]


    original_data["original_label"] = [column_encoder[i] for i in original_data["original_label"] ]

    attacked_data = attacked_data.rename(columns={'attacked_hypothesis': 'hypothesis','attacked_label':'label'})
    attacked_dataset = nlp.Dataset.from_pandas(attacked_data)

    attacked_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=attacked_dataset, label_map = reversed_encoder)
    
    original_data = original_data.rename(columns={'original_hypothesis': 'hypothesis','original_label':'label'})
    original_dataset = nlp.Dataset.from_pandas(original_data)

    original_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=original_dataset, label_map = reversed_encoder)
    
    print("------Attacked Dataset------")
    if not only_original:
      true_positives, false_negatives, pert_probs = evaluate_set(attacked_dataset,attack,logger)
      self.tp = true_positives
      self.fn = false_negatives
      self.adv_probs = pert_probs
      print("True positives :" + str(true_positives) + "\n")
      print("False_negatives :" + str(false_negatives) + "\n")

    if  not only_attack:
      print("------Original Dataset------")
      false_positives, true_negatives, or_probs = evaluate_set(original_dataset,attack,logger)
      print("False positives :" + str(false_positives) + "\n")
      print("True negative :" + str(true_negatives) + "\n")
      self.fp = false_positives
      self.tn = true_negatives
      self.or_probs = or_probs

# Attack whole eval with random

In [ ]:
def return_multi_nli_eval_dict(dataset):

  train_data = nlp.load_dataset(dataset)['train']

  valid_dict = {'premise':[],'hypothesis':[]}

  perc_ = math.ceil(len(train_data)*0.15)
  print("----EVAL DATASET USING : " + str(perc_) + "\n")

  eval_indices = np.sort(np.random.RandomState(21).choice(np.arange(len(train_data)),size=perc_,))

  valid_dict['premise'] = np.array(train_data['premise'])[eval_indices] 
  valid_dict['hypothesis'] = np.array(train_data['hypothesis'])[eval_indices]
  valid_dict['label'] = np.array(train_data['label'])[eval_indices]


  return valid_dict

In [ ]:
# Create Eval datasets
snli_dataset = nlp.load_dataset("snli")

#multi_eval_dataset = return_multi_nli_eval_dict("multi_nli")
snli_eval_dataset = snli_dataset['validation']
snli_eval_dataset = {'premise':snli_eval_dataset['premise'],'hypothesis':snli_eval_dataset['hypothesis'],'label':snli_eval_dataset['label']}

In [ ]:
from textattack.shared.utils import get_modifiable_indices

def parse_attack_from_recipe(recipe=None,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

  model = parse_model(model_from_file, model,model_from_huggingface,model_batch_size)

  recipe = eval(f"{ATTACK_RECIPE_NAMES[recipe]}.build(model)")
  recipe.goal_function.query_budget = float("inf")
  recipe.goal_function.model_batch_size = model_batch_size
  recipe.goal_function.model_cache_size = 2 ** 18
  recipe.constraint_cache_size = 2 ** 18
  return recipe
    
def create_snli_dict():
    dictionary = {}
    dictionary["premise"] = []
    dictionary["original_hypothesis"] = []
    dictionary["attacked_hypothesis"] = []
    dictionary["label"] = []
    dictionary["original_label"] = []
    dictionary["attacked_label"] = []
    dictionary['modifiable_indices'] = []
    dictionary['modified_indices'] = []

    return dictionary

def create_csv_file(dictionary,filename):
    df = pd.DataFrame(dictionary)
    df.to_csv(filename, encoding='utf-8', index=False)
    return df

def attack_recipe(recipe,dataset,data_name,folder,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):
     
  # Get attack
  attack = parse_attack_from_recipe(recipe,model_from_file, model,model_from_huggingface,model_batch_size)
    
  #Remove untified instances
  to_remove = []
  for index,x in dataset.iterrows():
    if x['label'] == -1:
      to_remove.append(index)

  dataset = dataset.drop(to_remove)
  dataset = nlp.Dataset.from_pandas(dataset)

  dataset = textattack.datasets.HuggingFaceNlpDataset(
          name=data_name, shuffle=False, split=None, dataset=dataset, label_map = reversed_encoder)
    
  num_remaining_attacks = len(dataset)
  worklist = deque(range(0, len(dataset)))
  start_time = time.time()
  worklist_tail = worklist[-1]

  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)
  attack_log_manager = parse_log_manager()


  num_results = 0
  num_failures = 0
  num_successes = 0

  load_time = time.time()

  output_attack_dir = os.path.join(folder, "attack")
  output_attack_dir = os.path.normpath(output_attack_dir)
  if not os.path.exists(output_attack_dir):
    os.makedirs(output_attack_dir)

  print("\nAttacks will be saved at " + output_attack_dir + "\n")
  attacked_dict = create_snli_dict()

  for result in attack.attack_dataset(dataset, indices=worklist):

    modifiable_indices = get_modifiable_indices(result.original_result.attacked_text,attack.pre_transformation_constraints)
    prem = result.original_result.attacked_text._text_input['premise']
    hypothesis = result.original_result.attacked_text._text_input['hypothesis']
    attacked_dict['premise'].append(prem)
    attacked_dict['original_hypothesis'].append(hypothesis)
    result.change_word_numbers = len(hypothesis.split())
    attacked_dict['label'].append(result.original_result.ground_truth_output)
    attacked_dict['original_label'].append(result.original_result._processed_output[0])
    attacked_dict['modifiable_indices'].append(list(modifiable_indices))

    if isinstance(result, textattack.attack_results.SkippedAttackResult):
        attacked_dict['attacked_hypothesis'].append(hypothesis)
        attacked_dict['attacked_label'].append(result.original_result._processed_output[0])
        attacked_dict['modified_indices'].append([])

    else:

        hypothesis = result.perturbed_result.attacked_text._text_input['hypothesis']
        attacked_dict['attacked_hypothesis'].append(hypothesis)
        attacked_dict['attacked_label'].append(result.perturbed_result._processed_output[0])
        attacked_dict['modified_indices'].append(list(result.perturbed_result.attacked_text.attack_attrs['modified_indices']))

   
    attack_log_manager.log_result(result)

    print("\n")
    pbar.update(1)
    
    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

  pbar.close()

  # Enable summary stdout
  
  attack_log_manager.log_summary()
  attack_log_manager.flush()
  print()
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  filename = output_attack_dir + "/" + recipe + "eval_whole" 

  create_csv_file(attacked_dict,filename+"_attacked.csv")

In [ ]:
#multi_eval_dataset = pd.DataFrame.from_dict(multi_eval_dataset)
snli_eval_dataset = pd.DataFrame.from_dict(snli_eval_dataset)
classifier = '../TrainedBertModels/bert-base-uncased-snli'
attack_recipe("random_nli",snli_eval_dataset,"snli",classifier,model_from_file=None, model=None,model_from_huggingface=classifier,model_batch_size=32)


NameError: ignored

#Get prectiles for delta tuning

In [ ]:
snli_dataset = nlp.load_dataset("snli")
#multi_dataset = nlp.load_dataset("multi_nli")
#multi_train_dataset = return_multi_nli_train_dict("multi_nli")
#multi_eval_dataset = return_multi_nli_eval_dict("multi_nli")
percentiles = [0,10,20,30,40,50,60,70,80,90,100]

In [ ]:
# SNLI 
frequencies_train = []
for sentence in snli_dataset['train']['hypothesis']:
  words = words_from_text(sentence)
  phi_values = get_phi_values(words,snli_freq_dict)
  frequencies_train += phi_values

print("SNLI frequency precentiles")
snli_precentiles = np.percentile(frequencies_train,percentiles)
print(snli_precentiles)

SNLI frequency precentiles
[ 0.69314718  6.3526294   7.83636976  8.92532142  9.80477158 10.72827684
 11.64400636 12.20202446 12.82231448 13.15694643 13.73591308]


In [ ]:
# MNLI
''' 
frequencies_train = []
for sentence in multi_train_dataset['hypothesis']:
  words = words_from_text(sentence)
  phi_values = get_phi_values(words,multi_freq_dict)
  frequencies_train += phi_values

print("MNLI frequency precentiles")
mnli_precentiles = np.percentile(frequencies_train,percentiles)
print(mnli_precentiles)
'''

' \nfrequencies_train = []\nfor sentence in multi_train_dataset[\'hypothesis\']:\n  words = words_from_text(sentence)\n  phi_values = get_phi_values(words,multi_freq_dict)\n  frequencies_train += phi_values\n\nprint("MNLI frequency precentiles")\nmnli_precentiles = np.percentile(frequencies_train,percentiles)\nprint(mnli_precentiles)\n'

# Tune delta for discete

In [ ]:
!ls ../TrainedBertModels/bert-base-uncased-snli/attack

alzantot_gpt2_2000_attacked.csv
freq_attacks_snli_FrequencyP_2000_attacked.csv
freq_attacks_snli_FrequencyR_2000_attacked.csv
prioritized_Priotirized_2000_attacked.csv
pwws_snli_2000_attacked.csv
random_nli_2000_attacked.csv
random_nlieval_whole_attacked.csv
textfooler_2000_attacked.csv


In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading SNLI training Corpus')
snli_freq_dict = corpus.load_csv('../snli_train_word_freq.csv')

def boostrap_auc(or_probs,adv_probs,times):

  np.random.seed(21)
  labels = np.concatenate((np.ones((len(adv_probs),)),np.zeros((len(adv_probs),))))
  au_scores = np.zeros(times)
  for i in range(times):
    or_sample_probs = np.random.choice(or_probs,len(adv_probs))
    probs = np.concatenate((np.array(adv_probs),np.array(or_sample_probs)))
    au_scores[i] = roc_auc_score(labels,probs)
  
  return np.mean(au_scores)

Try loading SNLI training Corpus


In [ ]:
tps_discrete = []
fns_discrete = []
fps_discrete = []
tns_discrete = []
f1_discrete = []
auc_scores = []

for i in snli_precentiles:
  print(str(i) + " delta \n")

  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/bert-base-uncased-snli',
        model_batch_size=32,
        delta = i,
        type = 'discrete',
        gamma = 1,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/bert-base-uncased-snli/attack/random_nlieval_whole_attacked.csv',
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False)
  
  tps_discrete.append(stat.tp)
  fns_discrete.append(stat.fn)
  fps_discrete.append(stat.fp)
  tns_discrete.append(stat.tn)

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_discrete.append(f1)
  
  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,1000)
  print("AUC score " + str(auc_score) + '\n')
  auc_scores.append(auc_score)
  
  

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/bert-base-uncased-snli


Try loading SNLI training Corpus
0.6931471805599453 delta 



textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH discrete--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
  0%|          | 0/3146 [00:00<?, ?it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 2332 / 814 / 3146: 100%|██████████| 3146/3146 [02:00<00:00, 26.11it/s]
textattack: Attack time: 120.51096606254578s
[Succeeded / Failed / Total] 0 / 4 / 4:   0%|          | 4/9842 [00:00<07:40, 21.36it/s]


True positives :2332

False_negatives :814

------Original Dataset------


[Succeeded / Failed / Total] 1913 / 7929 / 9842: 100%|██████████| 9842/9842 [06:25<00:00, 25.50it/s]
textattack: Attack time: 385.91540908813477s



False positives :1913

True negative :7929

F1 score 0.8532687651331718

AUC score 0.7241383056275055



In [ ]:
print("TPS")
print(tps_discrete)
print("\nFNS")
print(fns_discrete)
print("\nFPS")
print(fps_discrete)
print("\nTNS")
print(tns_discrete)
print("\nF1scores")
print(f1_discrete)
print("\nAUCscores")
print(auc_scores)

TPS
[1609, 2357, 2403, 2407, 2400, 2377, 2336, 2332, 2331, 2335, 2332]

FNS
[1537, 789, 743, 739, 746, 769, 810, 814, 815, 811, 814]

FPS
[18, 537, 952, 1260, 1550, 1703, 1875, 1867, 1885, 1911, 1913]

TNS
[9824, 9305, 8890, 8582, 8292, 8139, 7967, 7975, 7957, 7931, 7929]

F1scores
[0.9266613215111068, 0.9334871589085071, 0.9129653401797176, 0.8956843917966915, 0.8783898305084745, 0.8681599999999999, 0.8557924700574682, 0.8561000483065858, 0.8549478886859353, 0.8535299182092123, 0.8532687651331718]

AUCscores
[0.7470808496970289, 0.8291477288893273, 0.8075189907546652, 0.7864174396876892, 0.763756783000563, 0.747827040179378, 0.728454882010436, 0.7277609947989939, 0.7250769108008676, 0.7243598820629754, 0.7241383056275055]


In [ ]:
discrete_fp_rates = [fp / (fp+tn)  for fp,tn in zip(fps_discrete,tns_discrete)]
print("FPrate")
print(discrete_fp_rates)

discrete_tp_rates = [tp / (tp+fn)  for tp,fn in zip(tps_discrete,fns_discrete)]
print("\nTPrate")
print(discrete_tp_rates)
discrete_max_tp = np.argmax(discrete_tp_rates)
print("Maximum Tprate at delta :")
print(discrete_max_tp)
print("Best delta Value : ")
discrete_delta = snli_precentiles[discrete_max_tp]
print(discrete_delta)

FPrate
[0.001828896565738671, 0.05456208087787035, 0.09672830725462304, 0.12802275960170698, 0.15748831538305222, 0.17303393619183094, 0.19051005893111156, 0.1896972160130055, 0.19152611257874416, 0.1941678520625889, 0.1943710627921154]

TPrate
[0.5114431023521933, 0.7492053401144311, 0.7638270820089001, 0.7650985378258105, 0.7628734901462174, 0.7555626191989828, 0.7425301970756516, 0.7412587412587412, 0.7409408773045136, 0.742212333121424, 0.7412587412587412]
Maximum Tprate at delta :
3
Best delta Value : 
8.925321416943886


# Tune gamma (this will be done manually) take the best delta and try values for gamma then check the fpr

In [ ]:
#discrete_delta = 8.92532142

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading SNLI training Corpus')
snli_freq_dict = corpus.load_csv('../snli_train_word_freq.csv')

tps_discrete = []
fns_discrete = []
fps_discrete = []
tns_discrete = []
f1_discrete = []



for i in [0,0.2,0.4,0.6,0.8,1]:
  print(str(i) + " gamma \n")

  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/bert-base-uncased-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = i,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/bert-base-uncased-snli/attack/random_nlieval_whole_attacked.csv',
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=True)
  
  #tps_discrete.append(stat.tp)
  #fns_discrete.append(stat.fn)
  fps_discrete.append(stat.fp)
  tns_discrete.append(stat.tn)

  #true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  #predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  #f1 = f1_score(true_labels, predictions)
  #print("F1 score " + str(f1) + '\n')
  #f1_discrete.append(f1)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/bert-base-uncased-snli


Try loading SNLI training Corpus
0 gamma 



textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
  0%|          | 0/9842 [00:00<?, ?it/s]

...

--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 0 / 6 / 6:   0%|          | 6/9842 [00:00<05:13, 31.36it/s]

------Attacked Dataset------
------Original Dataset------


[Succeeded / Failed / Total] 0 / 9842 / 9842: 100%|██████████| 9842/9842 [04:20<00:00, 37.83it/s]
textattack: Attack time: 260.1848874092102s



False positives :0

True negative :9842



In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading SNLI training Corpus')
snli_freq_dict = corpus.load_csv('../snli_train_word_freq.csv')

tps_discrete = []
fns_discrete = []
fps_discrete = []
tns_discrete = []
f1_discrete = []



for i in [0.45,0.5,0.55,0.85,0.9,0.95]:
  print(str(i) + " gamma \n")

  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/bert-base-uncased-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = i,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/bert-base-uncased-snli/attack/random_nlieval_whole_attacked.csv',
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=True)
  
  #tps_discrete.append(stat.tp)
  #fns_discrete.append(stat.fn)
  fps_discrete.append(stat.fp)
  tns_discrete.append(stat.tn)

  #true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  #predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  #f1 = f1_score(true_labels, predictions)
  #print("F1 score " + str(f1) + '\n')
  #f1_discrete.append(f1)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/bert-base-uncased-snli


Try loading SNLI training Corpus
0.45 gamma 



textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 3 / 3 / 6:   0%|          | 6/9842 [00:00<05:14, 31.23it/s]

...

--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 2 / 4 / 6:   0%|          | 6/9842 [00:00<05:17, 30.98it/s]

------Attacked Dataset------
------Original Dataset------


[Succeeded / Failed / Total] 290 / 9552 / 9842: 100%|██████████| 9842/9842 [04:20<00:00, 37.71it/s]
textattack: Attack time: 260.9696726799011s



False positives :290

True negative :9552



In [ ]:
gamma_5 = 0.825
gamma_10 = 0.575 

# Defened against all attacks

In [ ]:
!ls ../TrainedBertModels/bert-base-uncased-snli/attack

alzantot_gpt2_2000_attacked.csv
freq_attacks_snli_FrequencyP_2000_attacked.csv
freq_attacks_snli_FrequencyR_2000_attacked.csv
prioritized_Priotirized_2000_attacked.csv
pwws_snli_2000_attacked.csv
random_nli_2000_attacked.csv
random_nlieval_whole_attacked.csv
textfooler_2000_attacked.csv


In [ ]:
attacks = ["random_nli_2000_attacked.csv","prioritized_Priotirized_2000_attacked.csv","textfooler_2000_attacked.csv","pwws_snli_2000_attacked.csv","alzantot_gpt2_2000_attacked.csv"]

### Discrete

In [ ]:
import tqdm
pbar = tqdm.tqdm(total=5, smoothing=0)
for i in range(5):
  pbar.update(1)
pbar.close()

100%|██████████| 5/5 [00:00<00:00, 7796.10it/s]


In [ ]:
tpr_discrete = []
fpr_discrete = []
f1_discrete = []
auc_scores = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/bert-base-uncased-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'discrete',
        gamma = None,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/bert-base-uncased-snli/attack/' + attack,
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_discrete.append(stat.tp/(stat.tp+stat.fn))
  fpr_discrete.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_discrete.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_scores.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/bert-base-uncased-snli


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH discrete--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
  0%|          | 0/603 [00:00<?, ?it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 886 / 432 / 1318: 100%|██████████| 1318/1318 [00:41<00:00, 32.06it/s]
textattack: Attack time: 41.110450744628906s
[Succeeded / Failed / Total] 0 / 7 / 7:   0%|          | 7/2000 [00:00<00:53, 37.05it/s]


True positives :886

False_negatives :432

------Original Dataset------


[Succeeded / Failed / Total] 246 / 1754 / 2000: 100%|██████████| 2000/2000 [00:53<00:00, 37.71it/s]
textattack: Attack time: 53.03335690498352s



False positives :246

True negative :1754

F1 score 0.8380315336837075

AUC score 0.7751192734082312



In [ ]:
print("TPrate")
print(tpr_discrete)
print("\nFPrate")
print(fpr_discrete)
print("\nF1 score")
print(f1_discrete)
print("\n AUC scores")
print(auc_scores)

TPrate
[0.7611940298507462, 0.7561224489795918, 0.6473004694835681, 0.7450856055802156, 0.6722306525037937]

FPrate
[0.123, 0.123, 0.123, 0.123, 0.123]

F1 score
[0.8999486916367369, 0.8785374405209116, 0.8055109070034442, 0.8440808469682387, 0.8380315336837075]

 AUC scores
[0.780489527349433, 0.8067579258642233, 0.7242080375986356, 0.8156854550330951, 0.7751192734082312]


### Continuous 10%

In [ ]:
tpr_cont10 = []
fpr_cont10 = []
f1_cont10 = []
auc_cont10 = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/bert-base-uncased-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = gamma_10,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/bert-base-uncased-snli/attack/' + attack,
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_cont10.append(stat.tp/(stat.tp+stat.fn))
  fpr_cont10.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_cont10.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_cont10.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/bert-base-uncased-snli


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 2 / 3 / 5:   1%|          | 5/603 [00:00<00:20, 29.44it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 693 / 625 / 1318: 100%|██████████| 1318/1318 [00:40<00:00, 32.60it/s]
textattack: Attack time: 40.4335560798645s
[Succeeded / Failed / Total] 0 / 7 / 7:   0%|          | 7/2000 [00:00<00:54, 36.26it/s]


True positives :693

False_negatives :625

------Original Dataset------


[Succeeded / Failed / Total] 201 / 1799 / 2000: 100%|██████████| 2000/2000 [00:53<00:00, 37.70it/s]
textattack: Attack time: 53.052998065948486s



False positives :201

True negative :1799

F1 score 0.8132911392405063

AUC score 0.7751192734082312



In [ ]:
print("TPrate")
print(tpr_cont10)
print("\nFPrate")
print(fpr_cont10)
print("\nF1 score")
print(f1_cont10)
print("\n AUC scores")
print(auc_cont10)

TPrate
[0.44776119402985076, 0.5897959183673469, 0.4055164319248826, 0.6106531388712746, 0.5257966616084977]

FPrate
[0.1005, 0.1005, 0.1005, 0.1005, 0.1005]

F1 score
[0.8707647628267182, 0.8564627469650082, 0.7477140482128014, 0.8153183775209608, 0.8132911392405063]

 AUC scores
[0.780489527349433, 0.8067579258642233, 0.7242080375986356, 0.8156854550330951, 0.7751192734082312]


## Continuous 5%

In [ ]:
tpr_cont5 = []
fpr_cont5 = []
f1_cont5 = []
auc_cont5 = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/bert-base-uncased-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = gamma_5,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/bert-base-uncased-snli/attack/' + attack,
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_cont5.append(stat.tp/(stat.tp+stat.fn))
  fpr_cont5.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_cont5.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_cont5.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/bert-base-uncased-snli


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 0 / 5 / 5:   1%|          | 5/603 [00:00<00:20, 28.74it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 399 / 919 / 1318: 100%|██████████| 1318/1318 [00:40<00:00, 32.43it/s]
textattack: Attack time: 40.646408796310425s
[Succeeded / Failed / Total] 0 / 7 / 7:   0%|          | 7/2000 [00:00<00:53, 37.11it/s]


True positives :399

False_negatives :919

------Original Dataset------


[Succeeded / Failed / Total] 113 / 1887 / 2000: 100%|██████████| 2000/2000 [00:52<00:00, 37.83it/s]
textattack: Attack time: 52.86208462715149s



False positives :113

True negative :1887

F1 score 0.7852684144818975

AUC score 0.7751192734082312



In [ ]:
print("TPrate")
print(tpr_cont5)
print("\nFPrate")
print(fpr_cont5)
print("\nF1 score")
print(f1_cont5)
print("\n AUC scores")
print(auc_cont5)

TPrate
[0.1857379767827529, 0.32040816326530613, 0.16666666666666666, 0.3709575142675967, 0.30273141122913505]

FPrate
[0.0565, 0.0565, 0.0565, 0.0565, 0.0565]

F1 score
[0.8620374600274098, 0.8289040193279156, 0.7111362351611079, 0.7735191637630663, 0.7852684144818975]

 AUC scores
[0.780489527349433, 0.8067579258642233, 0.7242080375986356, 0.8156854550330951, 0.7751192734082312]


# NWS and FGWS

In [ ]:
import scipy
import torch
import pandas as pd
from collections import deque
import corpus
import tqdm
import time
import textattack
from sklearn.metrics import accuracy_score

logger = textattack.shared.logger

                               
column_encoder = {"Entailment":0, "Neutral":1, "Contradiction":2}
reversed_encoder = {0:"Entailment", 1:"Neutral", 2:"Contradiction"}

def parse_log_manager():
    # Create logger
    attack_log_manager = textattack.loggers.AttackLogManager()

    '''
    # Get current time for file naming
    timestamp = time.strftime("%Y-%m-%d-%H-%M")

    # Get default directory to save results
    current_dir = os.path.dirname(os.path.realpath(__file__))
    outputs_dir = os.path.join(
        current_dir, os.pardir, os.pardir, os.pardir, "outputs", "attacks"
    )
    out_dir_txt = out_dir_csv = os.path.normpath(outputs_dir)

    # Get default txt and csv file names
    filename_txt = f"_{timestamp}.txt"
    filename_csv = f"_{timestamp}.csv"
    
    

    # in case directory doesn't exist
    if not os.path.exists(out_dir_txt):
        os.makedirs(out_dir_txt)
    if not os.path.exists(out_dir_csv):
        os.makedirs(out_dir_csv)

    attack_log_manager.add_output_file(os.path.join(out_dir_txt, filename_txt))
    '''
    #attack_log_manager.enable_visdom()
    #attack_log_manager.enable_wandb()
    attack_log_manager.enable_stdout()
    return attack_log_manager

def evaluate_set(attacked_dataset,attack,correct_labels,logger=True):

  num_remaining_attacks = len(attacked_dataset)
  worklist = deque(range(0, len(attacked_dataset)))
  start_time = time.time()
  worklist_tail = worklist[-1]

  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)
  #pbar = tqdm.autonotebook.tqdm(total=num_remaining_attacks)
  attack_log_manager = parse_log_manager()


  num_results = 0
  num_failures = 0
  num_successes = 0
  transform_labels = []

  load_time = time.time()

  probabilities = []
  for result in attack.attack_dataset(attacked_dataset, indices=worklist):

    #or_label = result.original_result.output
    #initial_score = result.original_result.raw_output[or_label]
    #pertubated_score = result.perturbed_result.raw_output[or_label]
    initial_score = 1-result.original_result.score
    pertubated_score = 1-result.perturbed_result.score
    probabilities.append(initial_score-pertubated_score)
    transform_labels.append(result.perturbed_result.output)

    if logger:
      attack_log_manager.log_result(result)

      print("\n")
    pbar.update(1)
    
    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )
    
  pbar.close()

  # Enable summary stdout
  
  if logger:
    attack_log_manager.log_summary()
    attack_log_manager.flush()
  print()
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  return num_successes, num_failures, probabilities, transform_labels



class EvalDefenseModel():

  def __init__(
      self,
      defense_from_class,
      model_from_file=None, 
      model=None,
      model_from_huggingface=None,
      model_batch_size=32,
      delta = 10,
      gamma = 1,
      type = 'discrete',
      freq_dict = None,
      data_file = None,
      data_name = "snli",
      candidates = 8,
      logger = True,
      only_attack = False,
      only_original = False,
      #indices_to_modify=None,
      #shifted_idxs=True,
    ):

    # Get attack
    attack = parse_defense_from_class(defense_from_class,freq_dict,delta,type,gamma,candidates,model_from_file, model,model_from_huggingface,model_batch_size)
    
    #Get datasets
    attacked_data = pd.read_csv(data_file,usecols=["premise",'attacked_hypothesis', 'label', 'modified_indices' , "attacked_label"])
    original_data = pd.read_csv(data_file,usecols=["premise",'original_hypothesis', "original_label"])
    or_labels = original_data["original_label"]
    

    to_remove = []
    for index,x in attacked_data.iterrows():
      if x['modified_indices'] == '[]':
        to_remove.append(index)
    attacked_data = attacked_data.drop(to_remove)
    correct_labels = attacked_data['label']
    
    

    attacked_data = attacked_data.drop(['modified_indices','label'],axis=1)

    attacked_data["attacked_label"] = [column_encoder[i] for i in attacked_data["attacked_label"] ]
    # Get original adv accuracy 
    self.adv_acc = accuracy_score(correct_labels,attacked_data["attacked_label"])

    original_data["original_label"] = [column_encoder[i] for i in original_data["original_label"] ]

    attacked_data = attacked_data.rename(columns={'attacked_hypothesis': 'hypothesis','attacked_label':'label'})
    attacked_dataset = nlp.Dataset.from_pandas(attacked_data)

    attacked_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=attacked_dataset, label_map = reversed_encoder)
    
    original_data = original_data.rename(columns={'original_hypothesis': 'hypothesis','original_label':'label'})
    original_dataset = nlp.Dataset.from_pandas(original_data)

    original_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=original_dataset, label_map = reversed_encoder)
    
    print("------Attacked Dataset------")
    if not only_original:
      true_positives, false_negatives, pert_probs, transform_labels = evaluate_set(attacked_dataset,attack,logger)
      self.tp = true_positives
      self.fn = false_negatives
      self.adv_probs = pert_probs

      print("True positives :" + str(true_positives) + "\n")
      print("False_negatives :" + str(false_negatives) + "\n")

      # Get transformed adv accuracy 
      self.transform_acc = accuracy_score(correct_labels,transform_labels)

    if  not only_attack:
      print("------Original Dataset------")
      false_positives, true_negatives, or_probs, _i10 = evaluate_set(original_dataset,attack,logger)
      print("False positives :" + str(false_positives) + "\n")
      print("True negative :" + str(true_negatives) + "\n")
      self.fp = false_positives
      self.tn = true_negatives
      self.or_probs = or_probs

In [ ]:
tpr_nws = []
fpr_nws = []
f1_nws = []
auc_nws = []
adv_acc_nws = []
tr_acc_nws =[]


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "NWS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/bert-base-uncased-snli',
        model_batch_size=32,
        delta = None,
        type = None,
        gamma = None,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/bert-base-uncased-snli/attack/' + attack,
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_nws.append(stat.tp/(stat.tp+stat.fn))
  fpr_nws.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_nws.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_nws.append(auc_score)

  adv_acc_nws.append(stat.adv_acc) 
  tr_acc_nws.append(stat.transform_acc)
  print("Adversarial Acc " + str(stat.adv_acc))
  print("Transform Acc " + str(stat.transform_acc))


In [ ]:
print("TP rates")
print(tpr_nws)
print("\nFP rates")
print(fpr_nws)
print("\nF1 scores")
print(f1_nws)
print("\nAuc scores")
print(auc_nws)
print("\nAdv_acc")
print(adv_acc_nws)
print("\nTransform_acc")
print(tr_acc_nws)

TP rates
[0.1600270087778528, 0.21604046242774566, 0.20631341600901917, 0.19808126410835214, 0.1493212669683258]

FP rates
[0.002, 0.0025, 0.0025, 0.002, 0.0005]

F1 scores
[0.7618320610687023, 0.7854330708661418, 0.7384786229872293, 0.7369392652759831, 0.7265128111938942]

Auc scores
[0.5535711569595703, 0.5647068921186391, 0.5888849497120502, 0.5856773881331879, 0.5528924500867611]

Adv_acc
[0.5928426738690075, 0.29190751445086704, 0.03945885005636979, 0.1100451467268623, 0.25452488687782804]

Transform_acc
[0.6988521269412559, 0.476878612716763, 0.2350620067643743, 0.29288939051918733, 0.38404977375565613]


In [ ]:
tpr_fgws = []
fpr_fgws = []
f1_fgws = []
auc_fgws = []
adv_acc_fgws = []
tr_acc_fgws = []

for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/bert-base-uncased-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'discrete',
        gamma = None,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/bert-base-uncased-snli/attack/' + attack,
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_fgws.append(stat.tp/(stat.tp+stat.fn))
  fpr_fgws.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_fgws.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_fgws.append(auc_score)
  
  adv_acc_fgws.append(stat.adv_acc) 
  tr_acc_fgws.append(stat.transform_acc)
  print("Adversarial Acc " + str(stat.adv_acc))
  print("Transform Acc " + str(stat.transform_acc))

In [ ]:
print("TP rates")
print(tpr_fgws)
print("\nFP rates")
print(fpr_fgws)
print("\nF1 scores")
print(f1_fgws)
print("\nAuc scores")
print(auc_fgws)
print("\nAdv_acc")
print(adv_acc_fgws)
print("\nTransform_acc")
print(tr_acc_fgws)

TP rates
[0.3565158676569885, 0.5447976878612717, 0.6223224351747464, 0.6602708803611738, 0.5005656108597285]

FP rates
[0.124, 0.124, 0.124, 0.124, 0.124]

F1 scores
[0.7447396386822529, 0.79963486992241, 0.7924016282225239, 0.8047772163527791, 0.7559870550161812]

Auc scores
[0.5437823792800182, 0.6296540704928749, 0.6998851814509722, 0.7317725613251532, 0.6196413450021498]

Adv_acc
[0.5928426738690075, 0.29190751445086704, 0.03945885005636979, 0.1100451467268623, 0.25452488687782804]

Transform_acc
[0.800810263335584, 0.7586705202312138, 0.6014656144306652, 0.7257336343115124, 0.6917420814479638]
